In [1]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
import torch.utils.data

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
IMG_SIZE = 32

In [4]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes):      
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=1, padding=1, bias = False)
        
    def forward(self,x):
        out = self.conv1(self.relu(self.bn1(x)))
        return torch.cat([x,out], 1)
        
class BottleneckBlock(nn.Module):
    def __init__(self, in_planes, out_planes):
        super(BottleneckBlock,self).__init__()
        
        # 중간 채널 수 = 4*k 로 설정
        inter_planes = out_planes * 4
        # 1x1 필터 적용
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, inter_planes, kernel_size=1, stride=1, padding=0, bias=False)
        # 3x3 필터
        self.bn2 = nn.BatchNorm2d(inter_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(inter_planes, out_planes, kernel_size=3, stride=1, padding=1, bias=False)
        
    def forward(self, x):
        out = self.conv1(self.relu(self.bn1(x)))
        out = self.conv2(self.relu(self.bn2(out)))
        
        # 입력값과 이번 레이어를 거친 출력값을 concat 해서 반환
        return torch.cat([x,out], 1)

In [5]:
class DenseBlock(nn.Module):
    def __init__(self, n_layers, in_planes, k, block):
        super(DenseBlock, self).__init__()
        self.layer = self.make_layer(block, in_planes, k, n_layers)
    
    # 덴스블록에 들어가는 레이어들을 생성하는 함수
    def make_layer(self, block, in_planes, k, n_layers):
        layers=[]
        # 레이어 개수만큼 맞는 블록타입의 레이어 생성
        for i in range(n_layers):
            layers.append(block(in_planes + i * k, k))
        return nn.Sequential(*layers)
    
    def forward(self,x):
        return self.layer(x)

In [11]:
class TransitionLayer(nn.Module):
    def __init__(self, in_planes, out_planes):
        super(TransitionLayer,self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1, padding=0, bias=False)
        
    def forward(self,x):
        out = self.conv1(self.relu(self.bn1(x)))
        return F.avg_pool2d(out, 2)

In [23]:
class DenseNet(nn.Module):
    def __init__(self, L, num_class, k=12, compression=0.5, bottleneck=True):
        super(DenseNet,self).__init__()
        num_of_blocks = 3
        n = int((L - num_of_blocks - 1) / num_of_blocks)
        
        # 첫 컨볼루션 채널 설정, 기본 16
        in_planes = 16 
        # DenseNet-BC 의 경우 대신 2*k 개로 설정
        if compression < 1:
            in_planes = 2 * k
        if bottleneck == True:
            in_planes = 2 * k 
            n = n // 2 
            block = BottleneckBlock 
        else :
            block = BasicBlock
        
        # DenseLayer 들어가기 전 Convolution
        self.conv1 = nn.Conv2d(3, in_planes, kernel_size=3, stride=1, padding=1, bias=False) 
    
        # DenseBlock 1
        self.db1 = DenseBlock(n, in_planes, k, block)
        in_planes = int(in_planes + n * k) 
        
        # TransitionLayer 1
        self.tl1 = TransitionLayer(in_planes, int(in_planes*compression))
        in_planes = int(in_planes * compression)
        
        # DenseBlock 2
        self.db2 = DenseBlock(n, in_planes, k, block)
        in_planes = int(in_planes + n * k)
        
        # TransitionLayer 2
        self.tl2 = TransitionLayer(in_planes, int(in_planes*compression))
        in_planes = int(in_planes * compression)
        
        # DenseBlock 3
        self.db3 = DenseBlock(n, in_planes, k, block)
        in_planes = int(in_planes + n * k)
        
        # Fully Connected Layer
        self.fc_layer = nn.Linear(in_planes, num_class)
        self.in_planes = in_planes
        
        # Weight Initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight.data)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_uniform_(m.weight.data)
        
    def forward(self,x):
        out = self.conv1(x)
        out = self.db1(out)
        out = self.tl1(out)
        out = self.db2(out)
        out = self.tl2(out)
        out = self.db3(out)
        out = F.avg_pool2d(out,8)
        out = out.view(-1, self.in_planes)
        out = self.fc_layer(out)
        return F.softmax(out, dim=1)

In [24]:
model = DenseNet(L=100, num_class=10, k=12, compression=0.5, bottleneck=True).to(device)
x = torch.randn(10, 3, IMG_SIZE, IMG_SIZE).to(device)
model(x)

tensor([[0.1049, 0.0266, 0.0853, 0.1072, 0.1611, 0.0620, 0.1630, 0.1031, 0.0494,
         0.1374],
        [0.0972, 0.0237, 0.0591, 0.0950, 0.1813, 0.0721, 0.1972, 0.1007, 0.0546,
         0.1192],
        [0.1412, 0.0304, 0.0743, 0.1147, 0.1331, 0.0738, 0.1326, 0.1317, 0.0512,
         0.1170],
        [0.1099, 0.0291, 0.0718, 0.0890, 0.1767, 0.0647, 0.1495, 0.1144, 0.0501,
         0.1449],
        [0.0872, 0.0276, 0.0738, 0.0971, 0.1512, 0.0780, 0.1783, 0.1168, 0.0511,
         0.1389],
        [0.1028, 0.0308, 0.0694, 0.1095, 0.1637, 0.0646, 0.1632, 0.1183, 0.0552,
         0.1224],
        [0.0899, 0.0304, 0.0757, 0.0870, 0.1627, 0.0670, 0.1889, 0.1139, 0.0415,
         0.1429],
        [0.0946, 0.0240, 0.0640, 0.0963, 0.1761, 0.0623, 0.1560, 0.1240, 0.0496,
         0.1532],
        [0.0986, 0.0306, 0.0652, 0.1101, 0.1399, 0.0855, 0.1553, 0.1166, 0.0513,
         0.1470],
        [0.0940, 0.0252, 0.0622, 0.0986, 0.1829, 0.0686, 0.1862, 0.1015, 0.0494,
         0.1314]], device='c